# モデル

このノートブックでは、前処理済みデータを用いてランダムフォレストによるモデルを構築し、評価します。

- 目的変数: `price_actual`
- モデル: ランダムフォレスト（RandomForestRegressor）
- 評価指標: RMSE
- ハイパーパラメータチューニング: GridSearchCV


## 1. ライブラリのインポートとデータ読み込み

In [1]:
# アクティベート
!source ../../.venv/bin/activate

.                   .DS_Store           .python-version     README.md
..                  .git                .venv               signate_smbc_202506
.cursor             .gitignore          pyproject.toml      uv.lock


In [2]:
# LightGBM特有のエラー対策
#!brew install libomp
#!pip uninstall lightgbm
#!pip install lightgbm

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import optuna

# データディレクトリ
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'
print(DATA_DIR)
# 前処理済みデータの読み込み
train = pd.read_csv(DATA_DIR / 'train_processed.csv')
test = pd.read_csv(DATA_DIR / 'test_processed.csv')

print('train shape:', train.shape)
print('test shape:', test.shape)

/Users/m0122wt/Desktop/02.プライベート/01.ノウハウ/07.データ分析/notebook/signate_smbc_202506/data
train shape: (26280, 122)
test shape: (8760, 121)


## 2. 特徴量・目的変数の設定

In [4]:
# 目的変数
target_col = 'price_actual'

# 説明変数（目的変数とtime列以外）
drop_cols = ['time', target_col] if target_col in train.columns else ['time']
feature_cols = [col for col in train.columns if col not in drop_cols]

X = train[feature_cols]
y = train[target_col] if target_col in train.columns else train.iloc[:, -1]  # 念のため

print('Features:', feature_cols)
print('Target:', target_col)
print('X shape:', X.shape)
print('y shape:', y.shape)

Features: ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other', 'generation_other_renewable', 'generation_solar', 'generation_waste', 'generation_wind_onshore', 'total_load_actual', 'valencia_temp', 'valencia_temp_min', 'valencia_temp_max', 'valencia_pressure', 'valencia_humidity', 'valencia_wind_speed', 'valencia_wind_deg', 'valencia_rain_1h', 'valencia_rain_3h', 'valencia_snow_3h', 'valencia_clouds_all', 'valencia_weather_id', 'valencia_weather_main', 'valencia_weather_description', 'valencia_weather_icon', 'madrid_temp', 'madrid_temp_min', 'madrid_temp_max', 'madrid_pressure', 'madrid_humidity', 'madrid_wind_speed', 'madrid_wind_deg', 'madrid_rain_1h', 'madrid_rain_3h', 'madrid_snow_3h', 'madrid_clouds_all', 'madrid_weather_id

## 3. 学習・検証データ分割

In [5]:
# ベイズ最適化によるLightGBMハイパーパラメータ探索
tscv = TimeSeriesSplit(n_splits=5)
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'verbose': -1,
        'random_state': 42
    }
    rmses = []
    for train_idx, valid_idx in tscv.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        train_data = lgb.Dataset(X_train, y_train)
        valid_data = lgb.Dataset(X_valid, y_valid, reference=train_data)
        model = lgb.train(params, train_data, valid_sets=[valid_data], num_boost_round=1000, callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
        y_pred = model.predict(X_valid)
        rmse = root_mean_squared_error(y_valid, y_pred)
        rmses.append(rmse)
    return np.mean(rmses)

## 4. モデルの学習とハイパーパラメータチューニング

In [6]:
%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)

[I 2025-06-21 15:38:56,283] A new study created in memory with name: no-name-9a113353-94ee-43fd-aaa3-614095e7dd30


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[157]	valid_0's rmse: 7.59273
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[65]	valid_0's rmse: 19.4568
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 11.4877
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 13.7993
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 15:39:00,672] Trial 0 finished with value: 11.696584746021035 and parameters: {'num_leaves': 36, 'learning_rate': 0.2525960635095359, 'feature_fraction': 0.7801401679873876, 'bagging_fraction': 0.7749046902733476, 'bagging_freq': 6, 'min_child_samples': 78}. Best is trial 0 with value: 11.696584746021035.


Early stopping, best iteration is:
[152]	valid_0's rmse: 6.14649
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[162]	valid_0's rmse: 7.23503
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[112]	valid_0's rmse: 19.2013
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 11.4537
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	valid_0's rmse: 12.3382
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 15:39:09,367] Trial 1 finished with value: 11.220347934092715 and parameters: {'num_leaves': 61, 'learning_rate': 0.08433672592747553, 'feature_fraction': 0.6276605863623268, 'bagging_fraction': 0.675321595802467, 'bagging_freq': 10, 'min_child_samples': 39}. Best is trial 1 with value: 11.220347934092715.


...

Early stopping, best iteration is:
[171]	valid_0's rmse: 5.83272
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[155]	valid_0's rmse: 7.30872
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[120]	valid_0's rmse: 19.4348
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 11.1587
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 13.3419
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 16:31:13,536] Trial 198 finished with value: 11.447906370233675 and parameters: {'num_leaves': 83, 'learning_rate': 0.07205404106205054, 'feature_fraction': 0.887436630158644, 'bagging_fraction': 0.8976908271849813, 'bagging_freq': 7, 'min_child_samples': 26}. Best is trial 178 with value: 10.962669375502992.


Early stopping, best iteration is:
[191]	valid_0's rmse: 5.99547
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[139]	valid_0's rmse: 7.33973
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[93]	valid_0's rmse: 19.5247
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 11.2319
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 13.346
Training until validation scores don't improve for 50 rounds


[I 2025-06-21 16:31:34,442] Trial 199 finished with value: 11.487922206748584 and parameters: {'num_leaves': 77, 'learning_rate': 0.08076731575862298, 'feature_fraction': 0.9453178528830499, 'bagging_fraction': 0.908627304514234, 'bagging_freq': 7, 'min_child_samples': 31}. Best is trial 178 with value: 10.962669375502992.


Early stopping, best iteration is:
[147]	valid_0's rmse: 5.99727
CPU times: user 1h 6min 4s, sys: 54min 46s, total: 2h 50s
Wall time: 52min 38s


In [7]:
print('LightGBM：ベイズ最適化の結果')
print('Best params:', study.best_params)
print('Best CV RMSE:', study.best_value)
best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42
})
train_data = lgb.Dataset(X, y)
best_model = lgb.train(best_params, train_data, num_boost_round=1000)

LightGBM：ベイズ最適化の結果
Best params: {'num_leaves': 72, 'learning_rate': 0.060106329525061244, 'feature_fraction': 0.6001190253500799, 'bagging_fraction': 0.9088733280452314, 'bagging_freq': 6, 'min_child_samples': 28}
Best CV RMSE: 10.962669375502992


履歴：ハイパーパラメータの設定  
ランダムフォレスト：グリッドサーチの結果  
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}  
Best score: 19.873329344971545  

LightGBM：ベイズ最適化の結果
Best params: {'num_leaves': 35, 'learning_rate': 0.07492759888976204, 'feature_fraction': 0.7423968779713126, 'bagging_fraction': 0.8599792857740763, 'bagging_freq': 1, 'min_child_samples': 13}
Best CV RMSE: 11.065542044290794

## 5. 最適なモデルでの予測と評価

In [8]:
# 特徴量重要度で下位20%を除外し再学習
importances = best_model.feature_importance(importance_type='gain')
threshold = np.percentile(importances, 20)
selected_features = [f for f, imp in zip(feature_cols, importances) if imp > threshold]
print('Selected features:', selected_features)
train_data_selected = lgb.Dataset(X[selected_features], y)
best_model = lgb.train(best_params, train_data_selected, num_boost_round=1000)

Selected features: ['generation_biomass', 'generation_fossil_brown_coal/lignite', 'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other', 'generation_other_renewable', 'generation_solar', 'generation_waste', 'generation_wind_onshore', 'total_load_actual', 'valencia_temp', 'valencia_temp_min', 'valencia_temp_max', 'valencia_pressure', 'valencia_humidity', 'valencia_wind_speed', 'valencia_wind_deg', 'valencia_clouds_all', 'valencia_weather_description', 'valencia_weather_icon', 'madrid_temp', 'madrid_temp_min', 'madrid_temp_max', 'madrid_pressure', 'madrid_humidity', 'madrid_wind_speed', 'madrid_wind_deg', 'madrid_clouds_all', 'madrid_weather_id', 'madrid_weather_icon', 'bilbao_temp', 'bilbao_temp_min', 'bilbao_temp_max', 'bilbao_pressure', 'bilbao_humidity', 'bilbao_wind_speed', 'bilbao_wind_de

## 6. テストデータへの予測と保存

In [9]:
# テストデータ予測と提出ファイル出力（フォーマット厳守）
X_test = test[selected_features]
test_pred = best_model.predict(X_test)
submission = test[['time']].copy()
submission['price_actual_pred'] = test_pred
assert submission.iloc[0,0] == '2018-01-01 00:00:00+01:00', '1行1列目が要件を満たしません'
submission.to_csv(DATA_DIR / 'submission_lightgbm.csv', index=False, header=False)
print('Saved: submission_lightgbm.csv')

Saved: submission_lightgbm.csv


In [10]:
submission

,time,price_actual_pred
0,2018-01-01 00:00:00+01:00,36.950365
1,2018-01-01 01:00:00+01:00,35.786768
2,2018-01-01 02:00:00+01:00,39.244586
3,2018-01-01 03:00:00+01:00,40.265419
4,2018-01-01 04:00:00+01:00,38.528570
...,...,...
8755,2018-12-31 19:00:00+01:00,70.622008
8756,2018-12-31 20:00:00+01:00,71.074073
8757,2018-12-31 21:00:00+01:00,65.407496
8758,2018-12-31 22:00:00+01:00,61.981665
